## Análisis para Generar Rutas

### Preprocesamiento

In [1]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
#import numpy as np
import difflib
from sklearn.feature_extraction.text import TfidfVectorizer


###### ArCo ARMADO DE LA BASE#######
import os
from typing import List
from datetime import datetime

# Establecer el directorio de trabajo - DNP
#os.chdir("C:/Users/supegui/Documents/Metodología ArCo/Python/Excel")

# Establecer el directorio de trabajo - Remoto
#os.chdir("C:/Backup - Archivos de trabajo/DNP/Automatización ArCo/Excel")

# Establecer el directorio de trabajo - Remoto
ruta = "Bases/2024 11 01 - Base Original.xlsx"


# Saca la lista de archivos del directorio de trabajo
#os.listdir()
# Obtener la fecha actual
fecha_actual = datetime.now().strftime("%Y %m %d")
#print(fecha_actual)

# Construir el nombre del archivo basado en la fecha actual
#nombre_archivo = f"{fecha_actual} Informe ArCo.xlsx"

# Construir el nombre del archivo cuando la fecha no es la actual. Aquí solo se modifica la fecha. ArCo Nal
#nombre_archivo = "2024 11 01 - Base Original Desc Completa.xlsx"


data_arco = pd.read_excel(ruta)

In [2]:
data_arco.head()

,Código,Sector,Entidad,Nombre del Instrumento,Descripción,Página Web,Antigüedad,Iteración,Emprendedores,Mipymes,...,1. Señale cuál o cuáles de las siguientes brechas,2. ¿En el diseño del instrumento se analizó el posible impacto diferenciado?,3. ¿El instrumento incorpora un enfoque interseccional?,Recursos fuente PGN,Recursos fuente SGR,Recursos fuente Cooperacion,Recursos fuente Esfuerzo Fiscal,Recursos fuente Parafiscales,Recursos fuente Otro,Estado
0,300,Trabajo,SERVICIO NACIONAL DE APRENDIZAJE - SENA,Programa de Formación Continua Especializada (...,El Programa de Formación Continua Especializad...,https://www.sena.edu.co/es-co/Empresarios/Pagi...,25.0,Primera iteración 2025 – Mes de reporte: Septi...,No,No,...,No,No,No,0,0,0,0,59708000000,0,Actualizado
1,301,Transporte,CORPORACION AUTONOMA REGIONAL DEL RIO GRANDE D...,Mantenimiento del Canal Navegable del Río Gran...,Cormagdalena adelantará d ragados y obras de d...,https://cormagdalena.gov.co/,28.0,Segunda iteración 2024 - Mes de reporte: Febre...,No,No,...,No,No,No,0,0,0,0,0,0,Desactualizado
2,310,Estadística,DEPARTAMENTO ADMINISTRATIVO NACIONAL DE ESTADI...,Información estadística de la temática económica,"El DANE ofrece diferentes investigaciones, vis...",https://www.dane.gov.co/index.php/estadisticas...,29.0,Primera iteración 2025 – Mes de reporte: Septi...,No,Si,...,No,No,No,40000000000,0,0,0,0,0,Actualizado
3,324,Hacienda,MINISTERIO DE HACIENDA Y CREDITO PUBLICO,Sistema Integrado de Información Financiera - ...,El Sistema Integrado de Información Financiera...,https://www.minhacienda.gov.co/webcenter/porta...,5.0,Primera iteración 2025 – Mes de reporte: Septi...,No,No,...,No,No,No,20713350446,0,0,0,0,0,Actualizado
4,343,Presidencia,AGENCIA PARA LA REINCORPORACION Y LA NORMALIZA...,Información del proceso de Reintegración y Rei...,"Brindar información de manera oportuna, veraz ...",http://www.reincorporacion.gov.co/es/la-reinte...,10.0,Primera iteración 2025 – Mes de reporte: Septi...,No,No,...,No,No,No,12525971945,0,0,0,0,0,Actualizado


In [3]:
# Get data information
data_arco.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 419 entries, 0 to 418
Data columns (total 71 columns):
 #   Column                                                                        Non-Null Count  Dtype         
---  ------                                                                        --------------  -----         
 0   Código                                                                        419 non-null    int64         
 1   Sector                                                                        418 non-null    object        
 2   Entidad                                                                       419 non-null    object        
 3   Nombre del Instrumento                                                        419 non-null    object        
 4   Descripción                                                                   419 non-null    object        
 5   Página Web                                                                    419 non-null  

In [5]:
# Selecting the relevant features for recommendation - Usuarios completos
selected_features = [
                 'Emprendedores',
                 #'Microempresa',
                 #'Pequeña Empresa',
                 #'Mediana Empresa',   
                 'Mipymes',
                 'Grandes empresas',
                 'Gobierno ',
                 'Academia',
                 'Entidades de soporte',
                 'Personas naturales',
                 ]

print(selected_features)

"""
# Selecting the relevant features for recommendation
selected_features = [
                 #'Emprendedores',
                 'Mipymes',
                 'Grandes empresas',
                 #'Gobierno ',
                 #'Academia',
                 #'Entidades de soporte',
                 #'Personas naturales'
                 ]
print(selected_features)
"""

['Emprendedores', 'Mipymes', 'Grandes empresas', 'Gobierno ', 'Academia', 'Entidades de soporte', 'Personas naturales']


"\n# Selecting the relevant features for recommendation\nselected_features = [\n                 #'Emprendedores',\n                 'Mipymes',\n                 'Grandes empresas',\n                 #'Gobierno ',\n                 #'Academia',\n                 #'Entidades de soporte',\n                 #'Personas naturales'\n                 ]\nprint(selected_features)\n"

In [6]:
# Replacing the null valuess with null string
for feature in selected_features:
    data_arco[feature] = data_arco[feature].fillna('')

In [8]:
# combining all the 5 selected features - Con todos los tipos de usuarios
combined_features = data_arco['Emprendedores'] + ' ' + data_arco['Entidades de soporte'] + ' ' + data_arco['Mipymes'] + ' ' + data_arco['Grandes empresas'] + ' ' + data_arco['Gobierno '] + ' ' + data_arco['Academia'] + ' ' + data_arco['Personas naturales'] 

combined_features



0      No Si No No No No No
1      No Si No Si No No No
2      No Si Si Si Si Si Si
3      No No No No Si No No
4      No Si No No Si Si Si
               ...         
414    No No No No No No Si
415    No Si Si No Si Si Si
416    No Si Si No Si Si Si
417    No No No No No No Si
418    Si Si Si Si Si No Si
Length: 419, dtype: object

### Construyendo el sistema de recomendación con el nombre del instrumento

In [9]:
# converting the text data to feature vectors
vectorizer = TfidfVectorizer()

feature_vectors = vectorizer.fit_transform(combined_features)

In [10]:
print(feature_vectors)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 818 stored elements and shape (419, 2)>
  Coords	Values
  (0, 0)	0.9867697053239923
  (0, 1)	0.1621281858740218
  (1, 0)	0.9302861121586437
  (1, 1)	0.3668347714227148
  (2, 0)	0.16669980365444861
  (2, 1)	0.9860076954372965
  (3, 0)	0.9867697053239923
  (3, 1)	0.1621281858740218
  (4, 0)	0.6054839362044375
  (4, 1)	0.795857526821466
  (5, 0)	0.8040883716653723
  (5, 1)	0.5945097901233671
  (6, 0)	0.9867697053239923
  (6, 1)	0.1621281858740218
  (7, 0)	0.9867697053239923
  (7, 1)	0.1621281858740218
  (8, 0)	0.9867697053239923
  (8, 1)	0.1621281858740218
  (9, 0)	0.9867697053239923
  (9, 1)	0.1621281858740218
  (10, 0)	0.8040883716653723
  (10, 1)	0.5945097901233671
  (11, 0)	0.6054839362044375
  (11, 1)	0.795857526821466
  (12, 0)	0.9867697053239923
  :	:
  (406, 1)	0.795857526821466
  (407, 0)	0.6054839362044375
  (407, 1)	0.795857526821466
  (408, 0)	0.9302861121586437
  (408, 1)	0.3668347714227148
  (409, 0)	0.60548393620

In [11]:
# getting the similarity scores using cosine similarity
similarity = cosine_similarity(feature_vectors, feature_vectors)

In [12]:
print(similarity)

[[1.         0.97745241 0.32435396 ... 0.52124267 1.         0.32435396]
 [0.97745241 1.         0.51678042 ... 0.68969209 0.97745241 0.51678042]
 [0.32435396 0.51678042 1.         ... 0.97633677 0.32435396 1.        ]
 ...
 [0.52124267 0.68969209 0.97633677 ... 1.         0.52124267 0.97633677]
 [1.         0.97745241 0.32435396 ... 0.52124267 1.         0.32435396]
 [0.32435396 0.51678042 1.         ... 0.97633677 0.32435396 1.        ]]


In [13]:
# creating a list with all the movie names given in the dataset

list_of_all_titles = data_arco['Nombre del Instrumento'].tolist()
print(list_of_all_titles)

['Programa de Formación Continua Especializada (PFCE)', 'Mantenimiento del Canal Navegable del Río Grande de la Magdalena', 'Información estadística de la temática económica', 'Sistema Integrado de Información Financiera - SIIF', 'Información del proceso de Reintegración y Reincorporación', 'Fomento de Innovación y Desarrollo Tecnológico en las Empresas', 'Servicio de asistencia jurídica y técnica para adelantar los procedimientos especiales agrarios', 'Servicio de Titulación colectiva Comunidades Negras- Constitución Resguardos Indígenas', 'Asesoría y capacitación:Plan Vial; inventario vial; categorización vial; presentación proyectos al SGR, FSSG, OxI; movilidad urbana sost, segur y salu', 'eKOGUI', 'Adecuación y asesoría para empresas colombianas del sector turismo', 'Colombia Investment Summit  - Macrorrueda Inversión Extranjera Directa', 'Servicio de ampliación R. Indígenas', 'Kit de Servicios para inversionistas extranjeros nuevos e instalados.', 'Servicio apoyo financiero inicia

In [14]:
# getting the movie name from the user
instrument_name = input(' Introducir el nombre del instrumento : ')
#instrument_name = 'Información estadística de la temática económica'


In [15]:
# finding the close match for the instrument name given by the user
find_close_match = difflib.get_close_matches(instrument_name, list_of_all_titles)
print(find_close_match)

['Programa de Formación Continua Especializada (PFCE)', 'Programa de Formación Especializada para la Economía Campesina (FEEC)']


In [16]:
from difflib import SequenceMatcher

def calculate_similarity(a, b):
    return SequenceMatcher(None, a, b).ratio()

# Ordenar los instrumentos según similitud con el nombre escogido
instrumentos_similares = sorted(
    list_of_all_titles,
    key=lambda x: calculate_similarity(instrument_name, x),
    reverse=True
)[:10]


In [17]:
print(instrumentos_similares)

['Programa de Formación Continua Especializada (PFCE)', 'Programa de Formación Especializada para la Economía Campesina (FEEC)', 'Programa de formación sobre mujer rural', 'Programa Nacional de Concertación Cultural', 'Programa de Fortalecimiento Productivo y Comercial a Pueblos Étnicos', 'Servicio de información "Conexión Justicia"', 'Programa de Educación Financiera Positiva', 'Programa Incentivo a medallas', 'Publicación de información de interés en salud', 'Programa de Modernización de Vehiculos de Carga']


### Análisis a partir de descripciones y features

In [20]:
# Fill missing values with empty strings
data_arco['Descripción'] = data_arco['Descripción'].fillna('')

# Convert text to lowercase
data_arco['Descripción'] = data_arco['Descripción'].str.lower()

# Vectorize the descriptions
vectorizer_desc = TfidfVectorizer()
description_vectors = vectorizer_desc.fit_transform(data_arco['Descripción'])

In [22]:
# Selected features
selected_features = ['Emprendedores', 'Microempresa', 'Pequeña Empresa', 'Mediana Empresa', 'Gobierno ', 'Academia', 'Personas naturales']

# Fill missing values with empty strings
for feature in selected_features:
    data_arco[feature] = data_arco[feature].fillna('')

# combining all the 5 selected features - Con todos los tipos de usuarios
combined_features = data_arco['Emprendedores'] + ' ' + data_arco['Microempresa'] + ' ' + data_arco['Pequeña Empresa'] + ' ' + data_arco['Mediana Empresa'] + ' ' + data_arco['Gobierno '] + ' ' + data_arco['Academia'] + ' ' + data_arco['Personas naturales'] 

# Vectorize the combined features
vectorizer_features = TfidfVectorizer()
feature_vectors = vectorizer_features.fit_transform(combined_features)

In [23]:
from sklearn.metrics.pairwise import cosine_similarity

# Description similarity matrix
description_similarity = cosine_similarity(description_vectors, description_vectors)

# Features similarity matrix
features_similarity = cosine_similarity(feature_vectors, feature_vectors)

In [24]:
# Define weights
weight_desc = 0.7
weight_features = 0.3

# Ensure both matrices are in the range [0, 1]
# If necessary, normalize the matrices
# For cosine similarity, values are already between -1 and 1, but we can adjust them to [0, 1]
description_similarity = (description_similarity + 1) / 2
features_similarity = (features_similarity + 1) / 2

# Combine the similarity matrices
combined_sim = (weight_desc * description_similarity) + (weight_features * features_similarity)

In [26]:
from difflib import SequenceMatcher

def find_closest_name(query, names):
    max_ratio = 0
    closest_name = ""
    for name in names:
        ratio = SequenceMatcher(None, query, name).ratio()
        if ratio > max_ratio:
            max_ratio = ratio
            closest_name = name
    return closest_name

def recommend_instruments(input_name, data_arco, combined_sim, top_n=5):
    # Find the closest instrument name
    closest_name = find_closest_name(input_name, data_arco['Nombre del Instrumento'])
    # Get the index of the selected instrument
    selected_index = data_arco[data_arco['Nombre del Instrumento'] == closest_name].index[0]
    # Get similarity scores
    sim_scores = list(enumerate(combined_sim[selected_index]))
    # Sort the scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    # Exclude the selected instrument
    sim_scores = [x for x in sim_scores if x[0] != selected_index]
    # Get top N indices
    top_indices = [i[0] for i in sim_scores[:top_n]]
    # Return recommended instruments
    return data_arco.iloc[top_indices][['Código', 'Nombre del Instrumento', 'Sector', 'Entidad']]

In [28]:
# Example usage
recommendations = recommend_instruments("Fábricas de Productividad y Sostenibilidad - Huila", data_arco, combined_sim, top_n=20)
print(recommendations)

    Código                             Nombre del Instrumento    Sector  \
9     3419  CELEBRACIÓN DEL DÍA DEL CAMPESINO DEL MUNICIPI...       NaN   
3     3321                                          CEmprende  Comercio   
4     3403  VERSIÓN 184 DE LA FERIA GANADERA, MERCADO GANA...       NaN   
7     3415    feria internacional de cafe cacao y agroturismo       NaN   
10    3428  Convocatoria Concertación Proyectos y Estimulo...       NaN   
5     3407  FERIA EQUINA GRADO B EN SU VERSIÓN 185, MERCAD...       NaN   
11    3760                                       Convocatoria       NaN   
12    3762  Gestión de la Información Estadística y Cartog...       NaN   
2     3308                                       Expovivienda       NaN   
8     3416  CONVOCATORIA PARA EL SECTOR TURISMO DEL DEPART...       NaN   
1     3304                 Cursos virtuales de Emprendimiento       NaN   
6     3410  Sistema de Información Turístico y Cultural de...       NaN   

                        

In [29]:
# Directorio DNP
recommendations.to_excel('C:/Users/supegui/Documents/Metodología ArCo/Python/Excel/recomendaciones_arco_huila.xlsx', index=False)

# Directorio remoto
#recommendations.to_excel('C:/Backup - Archivos de trabajo/DNP/Automatización ArCo/Excel/recomendaciones.xlsx', index=False)